<h1>
    Torch.AutoGrad에 대한 간단한 소개
</h1>
torch.autograd는 신경망 학습을 지원하는 PyTorch의 자동 미분 엔진입니다. 이 단원에서는 autograd가 신경망 학습을 어떻게 돕는지에 대한 개념적 이해를 할 수 있습니다.

<strong>배경(Background)</strong>

신경망(nn; Neural Network)은 어떤 입력 데이터에 대해 실행되는 중첩(nested)된 함수들의 모음입니다. 임 함수들은 PyTorch로 저장되는(가중치(weight)와 편향(bias)로 구성된 매개변수들로 정의됩니다.)

신경망을 학습하는 것은 2단계로 이루어집니다.

순전파(Forward Propagation): 순전파 단계에서, 신경망은 정답을 맞추기위해 최선의 추측을 합니다. 이렇게 추측을 하기 위해서 입력 데이터를 각 함수들에서 실행합니다.
역전파(Backward propagation): 역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여 매개변수들을 적절히 조절합니다. 출력으로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값(변화도(gradient))을 수집하고 경사하강법(gradient descent)을 사용하여 매개변수들을 최적합 합니다.

<strong>PyTorch에서 사용법</strong>

학습단계를 하나만 살펴보겠습니다. 여기에서는 torchvision에서 미리 학습된 resnet18모델을 불러옵니다. 3채널짜리 높이와 넓이가 64인 이미지 하나를 표현하는 무작위의 데이터 텐서를 생성하고, 이에 상응하는 label을 무작위 값으로 최적화합니다. 미리 학습된 모델의 정답은 (1,1000)의 모양을 갖는다.

In [5]:
import torch,torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)

다음으로, 입력데이터를 모델의 각 층에 통과시켜 예측값을 생성해보겠습니다. => <strong>순전파(forward Propagation)</strong>

In [7]:
prediction = model(data)

모델의 예측값과 그에 해당하는 label을 사용하여 loss(cost)를 계산합니다. 다음 단계는 신경망을 통해 이 loss(cost)를 역전파하는 것입니다. loss(cost)tensor에 .backward()함수를 호출하면 역전파가 시작됩니다. 그 다음 Autograd가 매개변수의 .grad속성에 모델의 각 매개변수에 대한 변화도(gradient)를 게산하고 저장합니다.

In [8]:
loss = (prediction  - labels).sum()
loss.backward() # 역전파 단계

다음으로, 옵티마이저(optimizer)를 불러옵니다. 이 예제에서는 learning rate 0.1과 momentum 0.9를 갖는 SGD입니다. 옵티마이저에 모델의 모든 매개변수를 등록합니다.

In [11]:
optimi = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

마지막으로 .step을 호출하여 경사하강법(gradient descent)을 시작합니다. optimizer는 .grad에 저장된 변화도에 따라 각 매개변수를 조정합니다.

In [12]:
optimi.step()

<h3>
    Autograd에서 미분(differentiation)
</h3>
autograd가 어떻게 변화도를 수집하는지 살펴보겠습니다. requires_grad=True를 갖는 2개의 텐서(tensor) a와 b를 만듭니다. requires_grad=True는 autograd에 모든 연산(operation)들을 추적해야 한다고 알려줍니다.

In [17]:
import torch

a=torch.tensor([2.,3.],requires_grad=True)
b=torch.tensor([6.,4.],requires_grad=True)

이제 a와 b로부터 새로운 텐서 Q를 만듭니다. 
$$ Q=3a^3-b^2$$

In [32]:
Q=3*a**3 - b**2

이제 a와 b가 모두 신경망(nn; Neural Network)의 매개변수이고, Q가 loss(coss)라고 가정해보겠습니다. 신경망을 학습 할 때, 아래와 같이 매개변수들에 대한 오차의 gradient를 구해야 합니다 즉, <br>$$\frac{∂Q}{∂a}=9a^2$$<br>$$\frac{∂Q}{∂b}=-2b$$<br><br>
Q에 대해서 .backward()를 호출할 때, autograd는 이러한 변화도들을 계산하고 이를 각 텐서의 .grad속성에 저장합니다.<br>
Q는 벡터(vector)이므로 Q.backward()에 gradient인자를 명시적으로 전달해야합니다. gradient는 Q와 같은 shape의 텐서로, Q자기 자신에 대한 변화도를 나타냅니다. 즉<br> $$\frac{dQ}{dQ}=1$$<br> 동일하게 Q.sum().backward()와 같이 Q를 스칼라 값으로 집계한 뒤 암시적으로 .backward()를 호출할 수도 있습니다.

In [33]:
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

In [34]:
print(9*a**2 == a.grad)

tensor([True, True])


In [35]:
print(-2*b == b.grad)

tensor([True, True])


<h3>
    선택적으로 읽기(Optional Teading)- autograd를 사용한 벡터 미적분(calculus)
</h3>

일반적으로 torch.autograd는 벡터-야코비안곱을 계산하는 엔진입니다. 이는 주어진 어떤 벡터에 대해 연산합니다.



<h3>
    미분을 하고 싶지 않은 매개변수설정
</h3>
미분을 하고 싶지 않은 매개변수 설정은 requires_grad=False로 설정하면 됩니다. 만약 미리학습된 신경망의 매개변수를 조정하고 싶지 않으면
for param in model.parameters():
    param.requires_grad=False
로 해주면 됩니다.